In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pwd

/content


In [3]:
WORKING_PATH = '/content/drive/MyDrive/NLP'

In [4]:
%cd {WORKING_PATH}

/content/drive/MyDrive/NLP


In [5]:
!ls

'1. Introduccion.pdf'				        __pycache__
'2.Introduccion al Procesado de Lenguaje Natural.pdf'   requirements.txt
 Clases						        sent.txt
 Datasets					        Soluciones
 datos_prepocesados.csv				        utils.py
 model.png					        w2v_model_fast.pkl
'Práctica final NLP.pdf'			        w2v_model_fast.pkl.wv.vectors_ngrams.npy
 Proyecto_final					        w2v_model.pkl


In [6]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
  Created wheel for pyDAWG: filename=pyDAWG-1.0.1-cp311-cp311-linux_x86_64.whl size=62783 sha256=be72c23ea0534f14239eef2447387790699818a37afdc16aa094ab90dbabfa54
  Stored in directory: /root/.cache/pip/wheels/b3/42/a5/2904aff781887b41600fdaf29aacb283e484d338c6932cdd25
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=95af2b4db50317fea743dc9743889363dd30a178fceda95f96640858528cd3fe
  Stored in directory: /root/.cache/pip/wheels/8f/a5/51/a5405e1da5d178491b79d12cc81b6cb9bb14fe2c8c632eba70
  Created wheel for docopt: 

In [7]:
!pip install pyLDAvis

In [8]:
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load
from google.colab import drive
from sklearn.feature_selection import chi2
import spacy
from textblob import TextBlob
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gemsimvis
pyLDAvis.enable_notebook()
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import inflect
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, SimpleRNN
from sklearn.metrics import confusion_matrix, classification_report


In [9]:
#Cargar el archivo de datos prepocesados
df = pd.read_csv('/content/drive/MyDrive/NLP/datos_prepocesados.csv')
print(df.head())

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   overall  verified   reviewTime      reviewerID        asin  \
0        5      True   09 1, 2016  A3CIUOJXQ5VDQ2  B0000530HU   
1        5      True  11 14, 2013  A3H7T87S984REU  B0000530HU   
2        1      True  08 18, 2013  A3J034YH7UG4KT  B0000530HU   
3        5     False   05 3, 2011  A2UEO5XR3598GI  B0000530HU   
4        5      True   05 6, 2011  A3SFRT223XXWF7  B00006L9LC   

                                               style     reviewerName  \
0  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...         Shelly F   
1  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...     houserules18   
2  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...             Adam   
3  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...           Rich K   
4                          {'Size:': ' 200ml/6.7oz'}  C. C. Christian   

                                          reviewText  \
0                   As advertised. Reasonably priced   
1  Like the oder and the feel when I put it on my...   
2 

In [10]:


#funcion para asignar etiquetas binarias
def asignar_etiqueta(overall):
  if overall >= 4: #ajustar el umbral
    return 1
  else:
    return 0

#Aplicamos la funcion para crear la etiqueta binaria
df['sentiment_label']=df['overall'].apply(asignar_etiqueta)




/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
def review_to_words(review):
  if pd.isna(review):
    return []
  text = BeautifulSoup(review).get_text()
  text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
  words = text.lower().split()
  words =[w for w in words if w not in stopwords.words('english')]
  words = [PorterStemmer().stem(w) for w in words]
  return words

df['reviewTextProcesado'] = df['reviewTextProcesado'].apply(review_to_words)


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
from imblearn.over_sampling import RandomOverSampler
#Preparacion de datos para Keras
#columna 'reviewTextProcesado'
#columna 'sentiment_label'(0,1)
#Convertir las listas de palabras a cadenas de texto
df['reviewTextProcesado'] = df['reviewTextProcesado'].apply(lambda x: ' '.join(x))

#Tokenizacion
tokenizer = Tokenizer(num_words=5000) #Ajuste de tamaño del vocabulario
tokenizer.fit_on_texts(df['reviewTextProcesado'])

X = tokenizer.texts_to_sequences(df['reviewTextProcesado'])
y = df['sentiment_label'].values

#Padding
max_words = 500 #longuitud maxima de las secuencias
X = pad_sequences(X, maxlen=max_words, padding = 'post', truncating='post')

#Division en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sobremuestreo (Oversampling)
# Aplicar RandomOverSampler SOLO al conjunto de entrenamiento
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, SimpleRNN
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report

# Parámetros comunes
embedding_size = 32
batch_size = 32
num_epochs = 10  # Puedes ajustar este valor

# Modelo LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(5000, embedding_size))  # vocabulary_size = 5000
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.build(input_shape=(None, max_words))
model_lstm.summary()
plot_model(model_lstm, show_shapes=True, show_layer_names=True)

model_lstm.fit(X_train_resampled, y_train_resampled,
              validation_split=0.1,
              shuffle=True,
              batch_size=batch_size,
              verbose=2,
              epochs=num_epochs)

# Evaluación LSTM
scores_lstm = model_lstm.evaluate(X_test, y_test, verbose=0)
print("Test accuracy (LSTM):", scores_lstm[1])
predictions_lstm = model_lstm.predict(X_test)
y_pred_lstm = (predictions_lstm >= 0.5).astype(int)
print('LSTM\n')
print(confusion_matrix(y_test, y_pred_lstm))
print(classification_report(y_test, y_pred_lstm))


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 32)             │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          53,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 213,301 (833.21 KB)

 Trainable params: 213,301 (833.21 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
225/225 - 224s - 997ms/step - accuracy: 0.5537 - loss: 0.6883 - val_accuracy: 0.0000e+00 - val_loss: 0.7676
Epoch 2/10
225/225 - 264s - 1s/step - accuracy: 0.5557 - loss: 0.6880 - val_accuracy: 0.0000e+00 - val_loss: 0.7735
Epoch 3/10
225/225 - 223s - 990ms/step - accuracy: 0.5556 - loss: 0.6871 - val_accuracy: 0.0000e+00 - val_loss: 0.8181
Epoch 4/10
225/225 - 205s - 912ms/step - accuracy: 0.5556 - loss: 0.6875 - val_accuracy: 0.0000e+00 - val_loss: 0.8361
Epoch 5/10
225/225 - 219s - 973ms/step - accuracy: 0.5556 - loss: 0.6875 - val_accuracy: 0.0000e+00 - val_loss: 0.7907
Epoch 6/10
225/225 - 218s - 971ms/step - accuracy: 0.5556 - loss: 0.6874 - val_accuracy: 0.0000e+00 - val_loss: 0.8151
Epoch 7/10
225/225 - 176s - 784ms/step - accuracy: 0.5556 - loss: 0.6873 - val_accuracy: 0.0000e+00 - val_loss: 0.8365
Epoch 8/10
225/225 - 207s - 922ms/step - accuracy: 0.5556 - loss: 0.6873 - val_accuracy: 0.0000e+00 - val_loss: 0.8111
Epoch 9/10
225/225 - 179s - 795ms/step - accuracy: 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional  # Import Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report

#Modelo GRU con Dropout y regularizacion L2
model_gru = Sequential()
model_gru.add(Embedding(5000, embedding_size, input_length=max_words))
model_gru.add(Bidirectional(GRU(100, dropout=0.3, recurrent_dropout=0.3)))
model_gru.add(Dense(64,activation='relu', kernel_regularizer=l2(0.01)))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(1, activation='sigmoid'))

model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru(np.zeros((1, max_words)))
model_gru.summary()
plot_model(model_gru, show_shapes=True, show_layer_names=True)

model_gru.fit(X_train_resampled, y_train_resampled,
              validation_split=0.1,
              shuffle=True,
              batch_size=batch_size,
              verbose=2,
              epochs=num_epochs)

#Evaluacion de GRU
scores_gru = model_gru.evaluate(X_test, y_test, verbose=0)
print("Test accuracy (GRU):", scores_gru[1])
predictions_gru = model_gru.predict(X_test)
y_pred_gru = (predictions_gru >= 0.5).astype(int)
print('GRU\n')
print(confusion_matrix(y_test, y_pred_gru))
print(classification_report(y_test, y_pred_gru))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (1, 500, 32)                │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (1, 200)                    │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (1, 64)                     │          12,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (1, 64)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (1, 1)                      │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 253,329 (989.57 KB)

 Trainable params: 253,329 (989.57 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
225/225 - 381s - 2s/step - accuracy: 0.8492 - loss: 0.5308 - val_accuracy: 0.9787 - val_loss: 0.1471
Epoch 2/10
225/225 - 330s - 1s/step - accuracy: 0.9862 - loss: 0.0714 - val_accuracy: 1.0000 - val_loss: 0.0247
Epoch 3/10
225/225 - 328s - 1s/step - accuracy: 0.9957 - loss: 0.0333 - val_accuracy: 1.0000 - val_loss: 0.0170
Epoch 4/10
225/225 - 381s - 2s/step - accuracy: 0.9980 - loss: 0.0249 - val_accuracy: 1.0000 - val_loss: 0.0132
Epoch 5/10
225/225 - 380s - 2s/step - accuracy: 0.9987 - loss: 0.0158 - val_accuracy: 1.0000 - val_loss: 0.0093
Epoch 6/10
225/225 - 319s - 1s/step - accuracy: 0.9986 - loss: 0.0136 - val_accuracy: 1.0000 - val_loss: 0.0104
Epoch 7/10
225/225 - 318s - 1s/step - accuracy: 0.9976 - loss: 0.0186 - val_accuracy: 1.0000 - val_loss: 0.0087
Epoch 8/10
225/225 - 317s - 1s/step - accuracy: 0.9992 - loss: 0.0116 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 9/10
225/225 - 324s - 1s/step - accuracy: 0.9993 - loss: 0.0087 - val_accuracy: 1.0000 - val_loss:

In [ ]:
########RNN+Word2vecEmdedding

In [14]:
import gensim
import multiprocessing as mp
import numpy as np  # Import numpy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.datasets import imdb  # Import imdb dataset
from tensorflow.keras.utils import plot_model

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
def generate_tokenizer(train_df):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(train_df)
  vocab_size = len(tokenizer.word_index) + 1
  print(f'Total words:{vocab_size}')
  return tokenizer, vocab_size

def generate_word2vec(train_df):
  documents = [_text.split() for _text in train_df]
  w2v_model= gensim.models.word2vec.Word2Vec(
      vector_size=W2V_SIZE,
      window=W2V_WINDOW,
      min_count=W2V_MIN_COUNT,
      workers=mp.cpu_count()
  )

  w2v_model.build_vocab(documents)

  words = list(w2v_model.wv.key_to_index.keys())
  vocab_size = len(words)
  print(f'Vocab size: {vocab_size}')
  print(f'Word2vec sample: {words[:10]}')
  w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH, report_delay=1)

  return w2v_model

def generate_embedding(word2vec_model, vocab_size, tokenizer):
  embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
  for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
      embedding_matrix[i] = word2vec_model.wv[word]

  return Embedding(
      vocab_size,
      W2V_SIZE,
      weights=[embedding_matrix],
      input_length=SEQUENCE_LENGTH,
      trainable=False,
  )


  w2v_model.build_vocab(documents)

  words = list(w2v_model.wv.key_to_index.keys())
  vocab_size = len(words)
  print(f'Vocab size: {vocab_size}')
  print(f'Word2vec sample: {words[:10]}')
  w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH, report_delay=1)

  return w2v_model



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Parámetros
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 5
W2V_MIN_COUNT = 2
max_words = 500 #maximo de palabras
SEQUENCE_LENGTH = max_words
vocabulary_size = 5000

#Cargar datos IMDB
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=None)#Cargamos vocabulario completo

word2id= imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

#Conertir a texto

X_train_words_text = [' '.join([id2word.get(i, ' ')for i in seq]).strip() for seq in X_train]
X_test_words_text = [' '.join([id2word.get(i, ' ')for i in seq]).strip() for seq in X_test]

import pandas as pd
X_train_words_text = pd.DataFrame(X_train_words_text, columns=['reviewTextProcesado'])
X_test_words_text = pd.DataFrame(X_test_words_text, columns=['reviewTextProcesado'])

#Prepocesamiento
tokenizer, vocab_size = generate_tokenizer(X_train_words_text.reviewTextProcesado)

# Combina train y test para Word2Vec
all_reviews = pd.concat([X_train_words_text.reviewTextProcesado, X_test_words_text.reviewTextProcesado])
word2vec_model = generate_word2vec(all_reviews)

embedding_layer = generate_embedding(word2vec_model, vocab_size, tokenizer)

X_train_words = pad_sequences(tokenizer.texts_to_sequences(X_train_words_text.reviewTextProcesado), maxlen=max_words, padding='post', truncating='post')
X_test_words = pad_sequences(tokenizer.texts_to_sequences(X_test_words_text.reviewTextProcesado), maxlen=max_words, padding='post', truncating='post')

#Modelo
model_custom = Sequential()
model_custom.add(embedding_layer)
model_custom.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences = True))#CapaLSTM con droupt
model_custom.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))#Otra capaLSTM
model_custom.add(Dense(1, activation="sigmoid"))

model_custom.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#Meto un lote de datos de ejemplo
dummy_input = np.zeros((1, SEQUENCE_LENGTH))
model_custom(dummy_input)

# No necesitas model_custom.build() si usas input_length en la capa Embedding
model_custom.summary() # Para ver la arquitectura del modelo
plot_model(model_custom, show_shapes=True, show_layer_names=True)

#y_train_index = np.array(y_train, dtype=int) # Asegurar el tipo correcto

#Entrenamiento
batch_size = 64
num_epochs = 10

history = model_custom.fit(X_train_words, y_train,
          validation_split=0.1,
          shuffle=True,
          batch_size=batch_size,
          verbose=1,
          epochs=num_epochs)

# Evaluación del modelo
scores = model_custom.evaluate(X_test_words, y_test, verbose=0)
print("Test accuracy:", scores[1])

predictions = model_custom.predict(X_test_words)
y_pred = (predictions >= 0.5).astype(int)

from sklearn.metrics import confusion_matrix, classification_report
print('\n')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Graficar historial de entrenamiento
plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validación')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Total words:88580
Vocab size: 61748
Word2vec sample: ['of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (1, 500, 300)               │      26,574,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (1, 500, 100)               │         160,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (1, 100)                    │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1, 1)                      │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,814,901 (102.29 MB)

 Trainable params: 240,901 (941.02 KB)

 Non-trainable params: 26,574,000 (101.37 MB)

Epoch 1/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1221s 3s/step - accuracy: 0.5084 - loss: 0.6913 - val_accuracy: 0.6688 - val_loss: 0.6213
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1252s 4s/step - accuracy: 0.6046 - loss: 0.6545 - val_accuracy: 0.5124 - val_loss: 0.6807
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1287s 4s/step - accuracy: 0.5262 - loss: 0.6788 - val_accuracy: 0.5064 - val_loss: 0.6863
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1252s 4s/step - accuracy: 0.6383 - loss: 0.6317 - val_accuracy: 0.5008 - val_loss: 0.6948
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1288s 4s/step - accuracy: 0.5819 - loss: 0.6542 - val_accuracy: 0.7364 - val_loss: 0.5682
Epoch 6/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 1275s 4s/step - accuracy: 0.7261 - loss: 0.5793 - val_accuracy: 0.7644 - val_loss: 0.5273
Epoch 7/10
225/352 ━━━━━━━━━━━━━━━━━━━━ 7:18 3s/step - accuracy: 0.7355 - loss: 0.5639